# 라이브러리 불러오기

In [1]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
     
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from pandas.io.html import read_html
import time
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from datetime import datetime, timedelta

# Scrapping(Departure)

In [2]:
#중앙 데이터프레임 만들기
df_Status = pd.DataFrame(columns=['항공사','목적지','계획','예상','출발','구분','현황','날짜'])

In [3]:
#날짜 리스트 생성
def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates

#2007.01.01~2021.07.31
dates = date_range("20190101", "20210731")

In [4]:
driver = webdriver.Chrome(executable_path='/Users/ikgyu/datacamp/project/chromedriver')

#암묵적으로 웹 자원 로드를 위해 10초까지 기다려 준다.
driver.implicitly_wait(10)

#url 접근
driver.get('https://www.airportal.go.kr/life/airinfo/RbHanFrm.jsp')
    
#'출발' click radio button
time.sleep(10)
accept_1 = driver.find_element_by_xpath("/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[1]/td[2]/input[2]")
accept_1.click()
#안 될때, Kernel -> Restart & Run all

In [5]:
for num in range(0,len(dates)):
    
    #execute_script 함수 사용
    driver.execute_script("document.getElementsByName('current_date')[0].value=\'"+dates[num]+"\'")
    
    #'검색' click
    accept_2 = driver.find_element_by_xpath("/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[4]/a/img")
    accept_2.click()
    
    #iframe 부르기
    iframes = driver.find_elements_by_tag_name('iframe')
    driver.switch_to.frame(iframes[0])
    html = driver.page_source
    
    #원래 프레임으로 돌아가기
    driver.switch_to.default_content()

    #html parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    #매개 데이터프레임 선언
    df_sample = pd.DataFrame()
    
    #요소별 리스트 생성
    airline_list = []
    destination_list = []
    plan_list = []
    prediction_list = []
    Start_list = []
    division_list = []
    Status_list = []

    #항공사 추출
    for airline in soup.find_all("td", {"width":"150"}):
        airline_list.append(airline.text.strip())
    
    #목적지 추출
    for destination in soup.find_all("td", {"width":"120"}):
        destination_list.append(destination.text.strip())
    
    #계획, 예상, 출발, 구분, 현황 추출
    list_ = []
    for info in soup.find_all("td", {"width":"50"}):
        list_.append(info.text)
    for i in range(0, len(list_)):
        if((i%5)==0):
            plan_list.append(list_[i])
        elif((i%5)==1):
            prediction_list.append(list_[i])
        elif((i%5)==2):
            Start_list.append(list_[i])
        elif((i%5)==3):
            division_list.append(list_[i])
        else:
            Status_list.append(list_[i])

    df_sample['항공사'] = Series(airline_list)
    df_sample['목적지'] = Series(destination_list)
    df_sample['계획'] = Series(plan_list)
    df_sample['예상'] = Series(prediction_list)
    df_sample['출발'] = Series(Start_list)
    df_sample['구분'] = Series(division_list)
    df_sample['현황'] = Series(Status_list)
    df_sample['날짜'] = dates[num]
    
    #중앙 데이터프레임에 합치기
    df_Status = pd.concat([df_Status,df_sample])
    
df_Status

<ipython-input-5-9ef9f00fa3a9>:57: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['항공사'] = Series(airline_list)
<ipython-input-5-9ef9f00fa3a9>:58: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['목적지'] = Series(destination_list)
<ipython-input-5-9ef9f00fa3a9>:59: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['계획'] = Series(plan_list)
<ipython-input-5-9ef9f00fa3a9>:60: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_sample['예상'] = Series(prediction_list)
<ipython-input-5-9ef9f00fa

,항공사,목적지,계획,예상,출발,구분,현황,날짜
0,아시아나항공,SEA(시애틀),00:05,00:10,00:25,화물,출발,20190101
1,에티하드,AUH(아부다비 국제공항),00:15,00:15,00:09,여객,출발,20190101
2,타이에어아시아엑스,DMK(방콕(돈므앙)),00:20,00:20,00:27,여객,출발,20190101
3,아틀라스화물항공,PVG(푸동),00:20,00:20,00:39,화물,출발,20190101
4,터키항공,IST(아타튀르크),00:30,00:30,00:30,여객,출발,20190101
...,...,...,...,...,...,...,...,...
207,대한항공,PVG(푸동),23:40,23:40,23:53,화물,출발,20210731
208,터키항공,IST(이스탄불),23:45,23:45,23:39,여객,출발,20210731
209,에티하드,AUH(아부다비 국제공항),23:45,23:45,23:45,여객,출발,20210731
210,에미레이트항공,DXB(두바이),23:55,23:55,23:56,여객,출발,20210731


In [6]:
#파일명 저장
df_Status.to_csv('Incheon_Departure_Status_Crawling_192021.csv',encoding='utf-8-sig')

# Scrapping(Arrival)


In [7]:
#중앙 데이터프레임 만들기
df_Status = pd.DataFrame(columns=['항공사','출발지','계획','예상','도착','구분','현황','날짜'])

In [8]:
#날짜 리스트 생성
def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates

#2007.01.01~2021.07.31
dates = date_range("20190101", "20210731")

In [9]:
driver = webdriver.Chrome(executable_path='/Users/ikgyu/datacamp/project/chromedriver')

#암묵적으로 웹 자원 로드를 위해 10초까지 기다려 준다.
driver.implicitly_wait(10)

#url 접근
driver.get('https://www.airportal.go.kr/life/airinfo/RbHanFrm.jsp')
    
#'도착' click radio button -> Default
time.sleep(10)
#안 될때, Kernel -> Restart & Run all

In [10]:
for num in range(0,len(dates)):
    
    #execute_script 함수 사용
    driver.execute_script("document.getElementsByName('current_date')[0].value=\'"+dates[num]+"\'")
    
    #'검색' click
    accept_2 = driver.find_element_by_xpath("/html/body/form/table/tbody/tr/td[2]/table/tbody/tr[3]/td/table/tbody/tr[2]/td/table/tbody/tr[2]/td[4]/a/img")
    accept_2.click()
    
    #iframe 부르기
    iframes = driver.find_elements_by_tag_name('iframe')
    driver.switch_to.frame(iframes[0])
    html = driver.page_source
    
    #원래 프레임으로 돌아가기
    driver.switch_to.default_content()

    #html parsing
    soup = BeautifulSoup(html, 'html.parser')
    
    #매개 데이터프레임 선언
    df_sample = pd.DataFrame()
    
    #요소별 리스트 생성
    airline_list = []
    DeparturePoint_list = []
    plan_list = []
    prediction_list = []
    Arrival_list = []
    division_list = []
    Status_list = []

    #항공사 추출
    for airline in soup.find_all("td", {"width":"150"}):
        airline_list.append(airline.text.strip())
    
    #출발지 추출
    for departure in soup.find_all("td", {"width":"120"}):
        DeparturePoint_list.append(departure.text.strip())
    
    #계획, 예상, 도착, 구분, 현황 추출
    list_ = []
    for info in soup.find_all("td", {"width":"50"}):
        list_.append(info.text)
    for i in range(0, len(list_)):
        if((i%5)==0):
            plan_list.append(list_[i])
        elif((i%5)==1):
            prediction_list.append(list_[i])
        elif((i%5)==2):
            Arrival_list.append(list_[i])
        elif((i%5)==3):
            division_list.append(list_[i])
        else:
            Status_list.append(list_[i])

    df_sample['항공사'] = Series(airline_list)
    df_sample['출발지'] = Series(DeparturePoint_list)
    df_sample['계획'] = Series(plan_list)
    df_sample['예상'] = Series(prediction_list)
    df_sample['도착'] = Series(Arrival_list)
    df_sample['구분'] = Series(division_list)
    df_sample['현황'] = Series(Status_list)
    df_sample['날짜'] = dates[num]
    
    #중앙 데이터프레임에 합치기
    df_Status = pd.concat([df_Status,df_sample])
    
df_Status

,항공사,출발지,계획,예상,도착,구분,현황,날짜
0,칼리타화물항공,JFK(뉴욕),00:05,00:29,00:29,화물,도착,20190101
1,비엣제트 항공,DAD(다낭),00:20,00:05,00:05,여객,도착,20190101
2,비엣제트 항공,CXR(깜라인(냐짱)),00:40,00:24,00:24,여객,도착,20190101
3,대한항공,PEK(베이징),01:00,00:32,00:32,여객,도착,20190101
4,폴라에어카고 월드와이드,CVG(신시내티),01:05,01:40,01:40,화물,도착,20190101
...,...,...,...,...,...,...,...,...
210,대한항공,LAX(로스앤젤레스),23:20,22:23,22:23,화물,도착,20210731
211,대한항공,LAX(로스앤젤레스),23:45,22:49,22:49,화물,도착,20210731
212,독일화물항공,NRT(나리타),23:50,23:42,23:42,화물,도착,20210731
213,에어브리지카고,SVO(셰레메티예보(모스크바)),23:50,23:13,23:13,화물,도착,20210731


In [11]:
#파일명 저장
df_Status.to_csv('Incheon_Arrival_Status_Crawling_192021.csv',encoding='utf-8-sig')